In [1]:
from whoosh.index import create_in
from whoosh.fields import *
import os
import re

In [2]:
schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT)
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = create_in("indexdir", schema)

In [3]:
import numpy as np
import datetime as dt

wd = 'downloaded_transcripts'

In [4]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [5]:
test = open(wd+"/AI_courses/A Nested Attention Neural Hybrid Model for Grammatical Error Correction _ ACL 2017-SI9cTcyNVew.en.vtt", "r", encoding='utf-8').read()
vtt0 = test.split('\n\n')

def cleanAutogenSubtitles(subtitles):
    line = [item for item in subtitles[2].split('\n') if item is not ' ']
    try:
        if(len(line[0].split(' --> ')[1]) > 12):
            cleaned = []
            for x in subtitles: 
                line = [item for item in x.split('\n') if item is not ' ']
                line[0] = line[0][:29]
                if(len(line) == 2 and len(cleanhtml(line[1])) == len(line[1])):
                    cleaned.append(line)
            subtitles = cleaned
    except:
        return subtitles
    
    return subtitles
            
# cleanAutogenSubtitles(vtt0)


In [6]:
next(os.walk('.'))[1]

subdir_ls = [(x[0],x[2]) for x in os.walk(wd) if len(x[2])>0] #only get directories with files

#part 0: Create dict of files
file_name_ls = []
file_content_ls = []
for i, subdir in enumerate(subdir_ls):
    print(str(i+1)+' of '+str(len(subdir_ls)), end='\r')
    subdir_path, subdir_files = subdir[0], subdir[1]
    file_name_ls += subdir_files
    for f in subdir_files:
        vtt0 = open(subdir_path+"/"+f, "r", encoding='utf-8').read()
        vtt1 = cleanAutogenSubtitles(vtt0.split('\n\n'))
        if vtt1 != ['']:
            vtt2 = []
            for x in vtt1:
                if(len(x)>0):
                    try:
                        tmp = x.split('\n')
                        timepoint = tmp[0].split(' --> ')
                        timepoint = timepoint[0] + '\t' + timepoint[1]   #convert to string
                        label = f + '\t' + timepoint
                        vtt2.append((label, ' '.join(tmp[1:]).lower()))
                    except:
                        continue

            vtt1 = vtt2
        file_content_ls.append(vtt1)
    
file_content_dict = dict(zip(file_name_ls,file_content_ls))

In [7]:
writer = ix.writer()
length = len(file_content_dict)
for i, f in enumerate(file_content_dict):
    print(f'{i+1} of {length}', end='\r')
    tmp = ''
    for caption in file_content_dict[f]:
        tmp += caption[1] + ' '
        
    writer.add_document(title=f, path="/"+f, content=tmp)

writer.commit()

In [8]:
from whoosh.qparser import QueryParser

In [21]:
def toseconds(timepoint):
    tmp = timepoint.split('.')[0].split(':')
    hours = int(tmp[0])
    minutes = int(tmp[1])
    seconds = int(tmp[2])
    return str(hours*360 + minutes*60 + seconds)

def searchCorpus(q):
    with ix.searcher() as searcher:
        query = QueryParser('content', ix.schema).parse(q)
        results = searcher.search(query, limit=10)
        results.fragmenter.charlimit = None
        # Show more context before and after
        results.fragmenter.surround = 400
        for hit in results:
            print(hit['title'][:-19] + '\n')

            filecontents = ''
            for line in file_content_dict[hit['title']]:
                filecontents += '\n' + line[0] + '\t' + line[1]
            
            snippets = hit.highlights('content', text=filecontents, top=5).split('...')
            snippets = [cleanhtml(x) for x in snippets]
            for snippet in snippets:
                surrounding_lines = snippet.split('\n')[1:-1]
                for i, line in enumerate(surrounding_lines):
                    tmp = line.split('\t')
                    print('\t' + tmp[3])
                    
                    if(i == 0):
                        video_id = tmp[0][-18:-7]
                        time_start = toseconds(tmp[1])
                
                try:
                    if(len(surrounding_lines) > 0):
                        print('\thttps://youtu.be/' + video_id + '?t=' + time_start + '\n')
                except:
                    continue
                
            
        return results

In [23]:
searchCorpus('machine learning algorithms')

Lecture 1.1 — Introduction What Is Machine Learning — [ Machine Learning _ Andrew Ng ]

	in this course i hope to teach you about various different types of learning algorithms
	https://youtu.be/PPLop4L2eGk?t=234

	these are other types of machine learning algorithms that we'll talk about later
	https://youtu.be/PPLop4L2eGk?t=276

	algorithms
	https://youtu.be/PPLop4L2eGk?t=291

	teaching about learning algorithms. it's like giving you a set of tools and
	https://youtu.be/PPLop4L2eGk?t=306

Tutorial 3.1 - Lorenzo Rosasco - Machine Learning Part 1

	there have been tons and tons and tons
	of algorithms developed.
	https://youtu.be/cyQZP23YbCY?t=166

	we're going to consider so-called regularization
	https://youtu.be/cyQZP23YbCY?t=364

	i mean, it's simple, but these are actually algorithms
	https://youtu.be/cyQZP23YbCY?t=1886

	ok.
	so we introduced a class of very simple algorithms.
	https://youtu.be/cyQZP23YbCY?t=3308

Lecture 8.6 - iCub Team - Overview of Research on the iCub Robot



<Top 10 Results for And([Term('content', 'machine'), Term('content', 'learning'), Term('content', 'algorithms')]) runtime=0.006772685001124046>